In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from dvclive import Live
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Device: {device}")


/Users/alice3e/.pyenv/versions/3.11.13/lib/python3.11/site-packages/dvclive/monitor_system.py:11: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  from pynvml import (


Device: mps


In [2]:
df = pd.read_csv('../data/balanced_animals_dataset.csv')
print(f"Dataset shape: {df.shape}")
print(df['scientific_name'].value_counts())


Dataset shape: (12000, 6)
scientific_name
Canis aureus              2400
Canis familiaris          2400
Canis familiaris dingo    2400
Canis latrans             2400
Canis lupus               2400
Name: count, dtype: int64


In [3]:
label_to_idx = {label: idx for idx, label in enumerate(df['scientific_name'].unique())}
idx_to_label = {idx: label for label, idx in label_to_idx.items()}
df['label'] = df['scientific_name'].map(label_to_idx)

print(f"Classes: {len(label_to_idx)}")
print(label_to_idx)


Classes: 5
{'Canis aureus': 0, 'Canis familiaris': 1, 'Canis familiaris dingo': 2, 'Canis latrans': 3, 'Canis lupus': 4}


In [4]:
train_df, val_df = train_test_split(
    df, 
    test_size=0.2, 
    stratify=df['label'], 
    random_state=42
)

print(f"Train: {len(train_df)}, Val: {len(val_df)}")
print(train_df['scientific_name'].value_counts())


Train: 9600, Val: 2400
scientific_name
Canis familiaris          1920
Canis familiaris dingo    1920
Canis aureus              1920
Canis latrans             1920
Canis lupus               1920
Name: count, dtype: int64


In [5]:
class AnimalDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.transform = transform
        self.base_path = Path('../animal_images')
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        species = row['scientific_name'].replace(' ', '_')
        img_path = self.base_path / species / f"{row['uuid']}.jpg"
        
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            image = Image.new('RGB', (224, 224), color=(0, 0, 0))
            
        label = row['label']
        
        if self.transform:
            image = self.transform(image)
            
        return image, label


In [6]:
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [7]:
train_dataset = AnimalDataset(train_df, transform=train_transform)
val_dataset = AnimalDataset(val_df, transform=val_transform)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")


Train batches: 300
Val batches: 75


In [8]:
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

for param in model.parameters():
    param.requires_grad = False

num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(label_to_idx))

model = model.to(device)


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)


In [10]:
def train_epoch_with_live(model, loader, criterion, optimizer, device, live):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(loader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    epoch_loss = running_loss / len(loader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc


In [11]:
def validate_with_live(model, loader, criterion, device, live):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Validation"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    epoch_loss = running_loss / len(loader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc, np.array(all_preds), np.array(all_labels)


In [24]:
num_epochs = 25
best_val_acc = 0.0
Path('../models').mkdir(exist_ok=True)

with Live(dir='../dvclive', save_dvc_exp=True) as live:
    
    live.log_params({
        'model': 'resnet50',
        'pretrained': 'IMAGENET1K_V1',
        'batch_size': batch_size,
        'learning_rate': 0.001,
        'num_epochs': num_epochs,
        'optimizer': 'Adam',
        'scheduler_step': 3,
        'scheduler_gamma': 0.1,
        'train_test_split': 0.2,
        'image_size': 224,
        'num_classes': len(label_to_idx)
    })
    
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        
        train_loss, train_acc = train_epoch_with_live(
            model, train_loader, criterion, optimizer, device, live
        )
        val_loss, val_acc, val_preds, val_labels = validate_with_live(
            model, val_loader, criterion, device, live
        )
        
        live.log_metric('train/loss', train_loss)
        live.log_metric('train/accuracy', train_acc)
        live.log_metric('val/loss', val_loss)
        live.log_metric('val/accuracy', val_acc)
        live.next_step()
        
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), '../models/best_model.pth')
            print(f"Best model saved with val_acc: {val_acc:.2f}%")
        
        scheduler.step()
    
    live.log_sklearn_plot(
        'confusion_matrix',
        val_labels,
        val_preds,
        name='confusion_matrix',
        labels=[idx_to_label[i] for i in range(len(idx_to_label))]
    )
    
    live.log_metric('best_val_accuracy', best_val_acc)



Epoch 1/1


Validation: 100%|██████████| 75/75 [00:15<00:00,  4.72it/s]


Train Loss: 1.1499, Train Acc: 55.01%
Val Loss: 0.9373, Val Acc: 64.92%
Best model saved with val_acc: 64.92%


ImportError: s3 is supported, but requires 'dvc-s3' to be installed

In [ ]:
print(classification_report(
    val_labels, 
    val_preds, 
    target_names=[idx_to_label[i] for i in range(len(idx_to_label))]
))


In [ ]:
# torch.save({
#     'model_state_dict': model.state_dict(),
#     'optimizer_state_dict': optimizer.state_dict(),
#     'label_to_idx': label_to_idx,
#     'idx_to_label': idx_to_label,
#     'best_val_acc': best_val_acc
# }, '../models/final_checkpoint.pth')

# print(f"Final checkpoint saved. Best validation accuracy: {best_val_acc:.2f}%")


Final checkpoint saved. Best validation accuracy: 67.46%


In [ ]:
# Path('../models').mkdir(exist_ok=True)

In [ ]:
# import json

# metrics = {
#     'best_val_accuracy': best_val_acc,
#     'final_train_accuracy': history['train_acc'][-1],
#     'final_val_accuracy': history['val_acc'][-1],
#     'final_train_loss': history['train_loss'][-1],
#     'final_val_loss': history['val_loss'][-1],
#     'num_epochs': num_epochs,
#     'num_classes': len(label_to_idx)
# }

# with open('../metrics.json', 'w') as f:
#     json.dump(metrics, f, indent=4)

# print("Metrics saved")


Metrics saved
